<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v2.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [29]:
# !pip install jsonlines
# import jsonlines

In [30]:
# #Install Polars with all optional dependencies.
# !pip install 'polars[all]'

In [45]:
#importing Libraries
# import polars as pl
from io import StringIO
import pandas as pd
import json
import os
import numpy as np
import csv
from datetime import datetime
import time
from datetime import timedelta

## Kaggle_data

In [4]:
# # 내 계정 API Token(json) 다운 받아서 업로드
# !pip install kaggle
# from google.colab import files
# files.upload()

In [5]:
# # Kaggle API를 사용하기 위해서 json파일 ~/.kaggle로 이동
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# #Permission Warning 방지
# !chmod 600 ~/.kaggle/kaggle.json

# # kaggle 데이터셋 API 주소 복사
# !kaggle competitions download -c otto-recommender-system

In [6]:
# !unzip /content/otto-recommender-system.zip

In [7]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test.csv',index = False)

###test_data, test_data_df

In [8]:
# # jsonl dataframe으로 읽기
# with open('test.jsonl', encoding='utf-8') as file:
#   test = [json.loads(line) for line in file]
# test=pd.DataFrame(test)
# test.head()

In [9]:
# # session별 events풀기
# test_df = pd.DataFrame()
# chunks = pd.read_json('test.jsonl', lines=True, chunksize=100_000)

# for chunk in chunks:
#     event_dict = {'session': [],'aid': [],'ts': [],'type': []}
#     for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
#         for event in events:
#             event_dict['session'].append(session)
#             event_dict['aid'].append(event['aid'])
#             event_dict['ts'].append(event['ts'])
#             event_dict['type'].append(event['type'])
#     chunk_session = pd.DataFrame(event_dict)
#     test_df = pd.concat([test_df, chunk_session])
            
# test_df = test_df.reset_index(drop=True)
# test_df

In [10]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test.csv',index = False)
# # !cp test.csv "drive/My Drive/OTTO/"
# # test.to_csv("test.csv", index = False)

In [11]:
# sample_sub.to_csv('/content/drive/My Drive/OTTO/sample_submission.csv',index = False)

## gdrive data

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# !unzip -qq "/content/drive/MyDrive/OTTO/train1.zip"

In [34]:
# train = pd.read_csv('train1.csv')
# train.session.nunique()
# train.type.nunique()
# train.shape

In [35]:
# train[["session", "ts"]].groupby("session").head()

In [36]:
# %%time
# train_df = train
# train_df["minutes"] = train_df[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
# train_df

In [37]:
# train_df.to_csv('/content/drive/My Drive/OTTO/train1_df.csv',index = False)
# train_df.to_csv("train1_df.csv", index = False)

In [38]:
train_df = pd.read_csv('/content/drive/My Drive/OTTO/train1_df.csv')
train_df.session.nunique()

163473

In [39]:
# 불가능
# train = pd.read_csv('/content/drive/My Drive/OTTO/train.csv')
# train.session.nunique()

## Create best-(sold, clicked) item item's list.

In [ ]:
# temp = train_df.groupby(['type','aid'])['session'].agg('count').reset_index()
# temp.columns = ['type','aid','count']
# temp.to_csv('/content/drive/My Drive/OTTO/temp.csv')
# order_num_df = temp.loc[(temp['type'] == 'orders'), ]
# order_num_df = order_num_df.sort_values(['count'],ascending=False).reset_index()
# order_num_df.to_csv('/content/drive/My Drive/OTTO/order_num_df.csv')
# clicks_num_df = temp.loc[(temp['type'] == 'clicks'), ]
# clicks_num_df = clicks_num_df.sort_values(['count'],ascending=False).reset_index()
# clicks_num_df.to_csv('/content/drive/My Drive/OTTO/clicks_num_df.csv')
# cart_num_df = temp.loc[(temp['type'] == 'cart'), ]
# cart_num_df = order_num_df.sort_values(['count'],ascending=False).reset_index()
# cart_num_df.to_csv('/content/drive/My Drive/OTTO/cart_num_df.csv')

In [12]:
temp = pd.read_csv('/content/drive/My Drive/OTTO/temp.csv')
order_num_df = pd.read_csv('/content/drive/My Drive/OTTO/order_num_df.csv')
clicks_num_df = pd.read_csv('/content/drive/My Drive/OTTO/clicks_num_df.csv')
cart_num_df = pd.read_csv('/content/drive/My Drive/OTTO/cart_num_df.csv')

In [13]:
order_num_df.aid = ' ' + order_num_df.aid.astype('str')
clicks_num_df.aid = ' ' + clicks_num_df.aid.astype('str')
cart_num_df.aid = ' ' + cart_num_df.aid.astype('str')
best_sold_list = order_num_df[:100].aid.sum()
best_clicked_list = clicks_num_df[:50].aid.sum()
best_shopping_list = cart_num_df[:20].aid.sum()

##test dataset
- 오래 본 아이템 추천하기

In [14]:
test = pd.read_csv('/content/drive/MyDrive/OTTO/test.csv')

In [ ]:
# test.to_csv("test.csv", index = False)

In [ ]:
# 6928123 4
test.shape

(6928123, 4)

In [52]:
train_df.session.nunique()-test.session.nunique()

-1508330

In [ ]:
test

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks
...,...,...,...,...
6928118,14571577,1141710,1662328774770,clicks
6928119,14571578,519105,1662328775009,clicks
6928120,14571579,739876,1662328775605,clicks
6928121,14571580,202353,1662328781067,clicks


In [ ]:
test.head()

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks


In [ ]:
# %%time
# test_df = test
# test_df["minutes"] = test[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)

In [ ]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test_df.csv',index = False)

In [15]:
test_df=pd.read_csv('/content/drive/My Drive/OTTO/test_df.csv')
test_df.head()

,session,aid,ts,type,minutes
0,12899779,59625,1661724000278,clicks,NaN
1,12899780,1142000,1661724000378,clicks,0.966233
2,12899780,582732,1661724058352,clicks,0.847450
3,12899780,973453,1661724109199,clicks,0.461150
4,12899780,736515,1661724136868,clicks,0.306333


In [16]:
# test_df = test_df.sort_values(['minutes'],ascending=False)
# test_df.head()

,session,aid,ts,type,minutes
30239,12905082,959154,1661726942565,clicks,10015.907300
4632,12900142,258596,1661724195582,clicks,10011.848333
17853,12902541,1544625,1661725403034,clicks,10007.667133
28235,12904668,510044,1661726708488,clicks,10002.046133
15991,12902229,1046138,1661725117749,clicks,9978.838333


In [19]:
# test_df_1 = test_df.copy()
# test_df_1.aid = ' ' + test_df_1.aid.astype('str')
# test_df_1

In [41]:
# a=test_df_1.iloc[10000][0]
# a=14571577
# test_df_1.loc[(test_df["session"] == a)]
# test_df_1[test_df_1['minutes'].isnull()]

,session,aid,ts,type,minutes
0,12899779,59625,1661724000278,clicks,NaN
5,12899780,1142000,1661724155248,clicks,NaN
16,12899781,918667,1662060160406,clicks,NaN
86,12899782,1007613,1661803953178,orders,NaN
97,12899783,1817895,1662041140398,clicks,NaN
...,...,...,...,...,...
6928118,14571577,1141710,1662328774770,clicks,NaN
6928119,14571578,519105,1662328775009,clicks,NaN
6928120,14571579,739876,1662328775605,clicks,NaN
6928121,14571580,202353,1662328781067,clicks,NaN


In [ ]:
14571577

In [18]:
# test_action_df = test_df.copy()
# # test_action_df = test_action_df.sort_values(['minutes'],ascending=False)
# test_action_df.aid = ' ' + test_df.aid.astype('str')
# test_action_df = test_action_df.groupby(['session','type'])['aid'].sum().reset_index()
# test_action_df
# test_action_df.to_csv('/content/drive/My Drive/OTTO/test_action_df.csv',index = False)

In [16]:
test_action_df=pd.read_csv('/content/drive/My Drive/OTTO/test_action_df.csv')

### only_type_df

In [17]:
# type: clicks
only_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ])
only_clicks_df
# next_carts_df['type'] = 'carts'
# next_carts_df

,session,type,aid
0,12899779,clicks,59625
1,12899780,clicks,1142000 582732 973453 736515 1142000
3,12899781,clicks,199008 194067 199008 199008 199008 199008 573...
5,12899782,clicks,603159 779477 1299062 602722 413962 975116 16...
7,12899783,clicks,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...,...
1948868,14571577,clicks,1141710
1948869,14571578,clicks,519105
1948870,14571579,clicks,739876
1948871,14571580,clicks,202353


In [18]:
# type: carts
only_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
only_carts_df
# next_orders_df['type'] = 'orders'
# next_orders_df

,session,type,aid
2,12899781,carts,199008
4,12899782,carts,1494780 834354 975116 127404 413962 595994 13...
10,12899786,carts,955252
12,12899787,carts,1682750 1682750 1682750
16,12899790,carts,1830166 1219653
...,...,...,...
1948716,14571430,carts,903014
1948730,14571443,carts,942326
1948774,14571486,carts,350578
1948788,14571499,carts,1132907


In [19]:
only_orders_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'orders'), ]).copy()
only_orders_df

,session,type,aid
6,12899782,orders,834354 1033148 595994 479970 1696036 1669402 ...
18,12899790,orders,1219653
22,12899793,orders,808362 1585431 1132001
30,12899799,orders,1191715 1325402 1191715 1388015 1309722 13254...
37,12899803,orders,929024 1651971 1684753
...,...,...,...
1946762,14569646,orders,483379
1947069,14569924,orders,1194467
1947364,14570190,orders,75811
1947700,14570500,orders,885614


In [23]:
# next_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ]).copy()
# next_clicks_df

## 생각해보기
### order: type_clicks & type_carts
### carts: type_carts & type_clicks
### clicks: type_carts & type_orders

In [20]:
# orders: type_carts & type_clicks
next_orders_df = pd.merge(only_clicks_df, only_carts_df[['session', 'aid']], on ='session', how = 'outer')
next_orders_df=next_orders_df.replace(np.nan, '')
next_orders_df["aid"] = next_orders_df["aid_x"] + next_orders_df["aid_y"]
next_orders_df = next_orders_df.drop(['aid_x', 'aid_y'], axis =1)
next_orders_df['type'] = 'orders'
next_orders_df

,session,type,aid
0,12899779,orders,59625
1,12899780,orders,1142000 582732 973453 736515 1142000
2,12899781,orders,199008 194067 199008 199008 199008 199008 573...
3,12899782,orders,603159 779477 1299062 602722 413962 975116 16...
4,12899783,orders,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...,...
1671328,14548635,orders,810041
1671329,14554536,orders,326274
1671330,14558693,orders,1437212
1671331,14563090,orders,1132194


In [21]:
# next_carts: type_click & type_order
next_carts_df = pd.merge(only_clicks_df, only_orders_df[['session', 'aid']], on ='session', how = 'outer')
next_carts_df=next_carts_df.replace(np.nan, '')
next_carts_df["aid"] = next_carts_df["aid_x"] + next_carts_df["aid_y"]
next_carts_df = next_carts_df.drop(['aid_x', 'aid_y'], axis =1)
next_carts_df['type'] = 'carts'
next_carts_df

,session,type,aid
0,12899779,carts,59625
1,12899780,carts,1142000 582732 973453 736515 1142000
2,12899781,carts,199008 194067 199008 199008 199008 199008 573...
3,12899782,carts,603159 779477 1299062 602722 413962 975116 16...
4,12899783,carts,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...,...
1670744,14548594,carts,873000
1670745,14548635,carts,810041
1670746,14549316,carts,150298
1670747,14559742,carts,490140


In [22]:
# next_clicks: type_carts & type_orders
next_clicks_df = pd.merge(only_carts_df, only_orders_df[['session', 'aid']], on ='session', how = 'outer')
next_clicks_df=next_clicks_df.replace(np.nan, '')
next_clicks_df["aid"] = next_clicks_df["aid_x"] + next_clicks_df["aid_y"]
next_clicks_df = next_clicks_df.drop(['aid_x', 'aid_y'], axis =1)
next_clicks_df['type'] = 'clicks'
next_clicks_df

,session,type,aid
0,12899781,clicks,199008
1,12899782,clicks,1494780 834354 975116 127404 413962 595994 13...
2,12899786,clicks,955252
3,12899787,clicks,1682750 1682750 1682750
4,12899790,clicks,1830166 1219653 1219653
...,...,...,...
245732,14562339,clicks,1112578 620545 1341391 1712154 303600
245733,14563634,clicks,756614
245734,14565992,clicks,395743
245735,14568250,clicks,422075 471339


In [23]:
recommend_df = pd.concat([next_orders_df, next_carts_df, next_clicks_df], axis =0)
recommend_df["session_type"] = recommend_df["session"].astype('str') + "_" + recommend_df["type"] 
recommend_df 

,session,type,aid,session_type
0,12899779,orders,59625,12899779_orders
1,12899780,orders,1142000 582732 973453 736515 1142000,12899780_orders
2,12899781,orders,199008 194067 199008 199008 199008 199008 573...,12899781_orders
3,12899782,orders,603159 779477 1299062 602722 413962 975116 16...,12899782_orders
4,12899783,orders,607638 1729553 255297 300127 1754419 1216820 ...,12899783_orders
...,...,...,...,...
245732,14562339,clicks,1112578 620545 1341391 1712154 303600,14562339_clicks
245733,14563634,clicks,756614,14563634_clicks
245734,14565992,clicks,395743,14565992_clicks
245735,14568250,clicks,422075 471339,14568250_clicks


In [24]:
sample_sub = pd.read_csv('/content/drive/My Drive/OTTO/sample_submission.csv')
sample_sub

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524
...,...,...
5015404,14571580_carts,129004 126836 118524
5015405,14571580_orders,129004 126836 118524
5015406,14571581_clicks,129004 126836 118524
5015407,14571581_carts,129004 126836 118524


In [42]:
session_type = sample_sub.session_type.str.split('_')
session_type

0          [12899779, clicks]
1           [12899779, carts]
2          [12899779, orders]
3          [12899780, clicks]
4           [12899780, carts]
                  ...        
5015404     [14571580, carts]
5015405    [14571580, orders]
5015406    [14571581, clicks]
5015407     [14571581, carts]
5015408    [14571581, orders]
Name: session_type, Length: 5015409, dtype: object

In [ ]:
session_type = session_type.apply(lambda x: pd.Series(x))
session_type

In [29]:
recommend_df[["session_type","aid"]]

,session_type,aid
0,12899779_orders,59625
1,12899780_orders,1142000 582732 973453 736515 1142000
2,12899781_orders,199008 194067 199008 199008 199008 199008 573...
3,12899782_orders,603159 779477 1299062 602722 413962 975116 16...
4,12899783_orders,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...
245732,14562339_clicks,1112578 620545 1341391 1712154 303600
245733,14563634_clicks,756614
245734,14565992_clicks,395743
245735,14568250_clicks,422075 471339


In [25]:
sample_sub_m = pd.merge(sample_sub, recommend_df[["session_type","aid"]], on = "session_type", how ="left")
sample_sub_m['next'] = sample_sub_m['aid'] + best_sold_list
sample_sub_m['next'].fillna(best_sold_list, inplace = True)
sample_sub_m['next'] = sample_sub_m['next'].str.strip()

In [26]:
sample_sub_m = sample_sub_m.drop(["labels", "aid"], axis = 1)
sample_sub_m.columns = ("session_type", "labels")
# sample_sub_m

In [28]:
sample_sub_m.to_csv('submission.csv', index=False)

In [27]:
sample_sub_m.to_csv('/content/drive/My Drive/OTTO/submission.csv', index=False)

In [ ]:
kaggle competitions submit -c otto-recommender-system -f submission.csv -m "Message"